In [ ]:
import json
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

# Features for text sentiment classification

In [ ]:
D = pd.read_csv('data/yelp_example_1_small.tsv', sep='\t')

In [ ]:
D.head()

## Terms and term frequencies

In [ ]:
import spacy
from collections import defaultdict

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
I = defaultdict(lambda: defaultdict(lambda: 0))

In [ ]:
records = []
rows = tqdm_notebook(list(D.iterrows()))
for i, row in rows:
    doc = nlp(row.content)
    for s, sent in enumerate(doc.sents):
        for t, token in enumerate(sent):
            record = {'doc': i, 'sentence': s, 'position': t}
            record['token'] = token.text
            record['lower'] = token.text.lower()
            record['lemma'] = token.lemma_
            record['pos'] = token.pos_
            record['alpha'] = token.is_alpha
            record['stop'] = token.is_stop
            record['doc_size'] = len(doc)
            record['sentence_size'] = len(sent)
            records.append(record)

## Store index

In [ ]:
import pymongo

In [ ]:
db = pymongo.MongoClient()['textsent']
yelp = db['yelp_simple']

In [ ]:
yelp.insert_many(records)

In [ ]:
m = {'$match': {'pos': {'$in': ['NOUN', 'ADJ']}}}
p = {'$project': {'_id': 0, 'doc': 1, 'sentence': 1, 'position': 1, 'lemma': 1}}
s = {'$sort': {'doc': 1, 'sentence': 1}}
g = {'$group': {'_id': '$doc', 'tokens': {'$push': '$lemma'}}}

In [ ]:
for record in yelp.aggregate([m, p, s, g], allowDiskUse=True):
    print(record)

In [ ]:
m = {'$match': {'pos': {'$in': ['NOUN', 'ADJ']}}}
g = {'$group': {'_id': {'doc': '$doc', 'size': '$doc_size', 'lemma': '$lemma'}, 'tf': {'$sum': 1}}}
h = {'$match': {'tf': {'$gte': 3}}}
s = {'$sort': {'tf': -1}}

In [ ]:
m = {'$match': {'pos': {'$in': ['NOUN', 'ADJ']}}}
g = {'$group': {'_id': '$lemma', 'docs': {'$addToSet': '$doc'}}}
p = {'$project': {'_id': 1, 'docs': {'$size': '$docs'}}}

In [ ]:
N = len(yelp.distinct('doc'))

In [ ]:
for record in yelp.aggregate([m, g, p], allowDiskUse=True):
    print(record['_id'], np.log(N / record['docs']))

## Add sentiment lexicon to the index

## Deal with the logical structure of sentences
### Take into account negation using a dependency parser